In [2]:
import cv2
from cv2 import aruco
import glob
import numpy as np

# Charuco board variables
CHARUCOBOARD_ROWCOUNT = 17
CHARUCOBOARD_COLCOUNT = 24
ARUCO_DICT = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
CHARUCO_BOARD = aruco.CharucoBoard(
    size=(CHARUCOBOARD_COLCOUNT, CHARUCOBOARD_ROWCOUNT),
    squareLength=0.0075,
    markerLength=0.005,
    dictionary=ARUCO_DICT)

# Glob images
images = glob.glob('./Images/*.jpg')

# Arrays to store Charuco corners and IDs for all images
allCorners = []
allIds = []

# Loop over images
for fname in images:
    # Load image
    img = cv2.imread(fname)
    # Resize image
    img = cv2.resize(img, (0, 0), fx=0.3, fy=0.3)
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find aruco markers in the query image
    corners, ids, rejected = aruco.detectMarkers(
        image=gray,
        dictionary=ARUCO_DICT)
    # If there are markers found, process them
    if ids is not None:
        # Refine detected markers
        corners, ids, rejectedImgPoints, recoveredIds = aruco.refineDetectedMarkers(
            image=gray,
            board=CHARUCO_BOARD,
            detectedCorners=corners,
            detectedIds=ids,
            rejectedCorners=rejected,
            cameraMatrix=None,
            distCoeffs=None)
        
        # Refine detected markers again!
        corners, ids, rejectedImgPoints, recoveredIds = aruco.refineDetectedMarkers(
            image=gray,
            board=CHARUCO_BOARD,
            detectedCorners=corners,
            detectedIds=ids,
            rejectedCorners=rejectedImgPoints,
            cameraMatrix=None,
            distCoeffs=None)
        
        # Interpolate charuco corners
        res, charucoCorners, charucoIds = aruco.interpolateCornersCharuco(
            markerCorners=corners,
            markerIds=ids,
            image=gray,
            board=CHARUCO_BOARD)
        
        # Add interpolated corners and ids to lists
        if charucoCorners is not None and charucoIds is not None:
            allCorners.append(charucoCorners)
            allIds.append(charucoIds)
            
        # Outline all of the markers detected in our image
        img = aruco.drawDetectedMarkers(
            image=img,
            corners=corners)
    # Save image
    cv2.imwrite('./Images/Output/' + fname[9:-4] + '_output.jpg', img)

# Create calibration
imsize = gray.shape
accuracy, cameraMatrix, distCoeffs, rvec, tvec = aruco.calibrateCameraCharuco(
    charucoCorners=allCorners,
    charucoIds=allIds,
    board=CHARUCO_BOARD,
    imageSize=imsize,
    cameraMatrix=None,
    distCoeffs=None)

# Print matrix and distortion coefficients
print(cameraMatrix)
print(distCoeffs)
print('rvecs: ',rvec)
print('tvecs:', tvec)

# save camera matrix and distortion coefficients
np.savez('./Images/Output/calibration.npz', cameraMatrix=cameraMatrix, distCoeffs=distCoeffs)

# Print accuracy
print(accuracy)

# Use the calibration to undistort an image
img = cv2.imread('./Images/M_K2_KneeArthroscopy_2023-09-08-145230__Surgeon_Still_001.jpg')
# Resize image
img = cv2.resize(img, (0, 0), fx=0.3, fy=0.3)
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(cameraMatrix, distCoeffs, (w,h), 1, (w,h))
dst = cv2.undistort(img, cameraMatrix, distCoeffs, None, newcameramtx)
# Crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
# Save image
cv2.imwrite('./Images/Output/undistorted.jpg', dst)

[[162.752863     0.         300.95088717]
 [  0.         162.34066574 146.67693793]
 [  0.           0.           1.        ]]
[[-0.48733348  0.38496066 -0.00495979  0.01150721 -0.16178221]]
rvecs:  (array([[-0.36896174],
       [ 0.16593303],
       [ 2.03183596]]), array([[-0.3402872 ],
       [ 0.17614105],
       [ 1.96926698]]), array([[-0.36304333],
       [ 0.0707981 ],
       [ 1.96810838]]), array([[-0.31498003],
       [ 0.11385127],
       [ 1.96204735]]), array([[-0.26085569],
       [ 0.11872492],
       [ 1.93645435]]), array([[-0.52204405],
       [ 0.95067195],
       [ 1.5754495 ]]), array([[-0.51595827],
       [ 1.02099083],
       [ 1.60878647]]), array([[-0.51099692],
       [ 1.00548084],
       [ 1.53145627]]), array([[-0.52229239],
       [ 1.07090742],
       [ 1.6344614 ]]), array([[-0.52736603],
       [ 1.13242379],
       [ 1.67961379]]), array([[ 0.17265673],
       [-0.66030834],
       [ 2.17835021]]), array([[ 0.18525126],
       [-0.69161635],
       [

True

In [16]:
import numpy as np
import glob
import cv2

# Glob images
images = glob.glob('./Images/*.jpg')

camera_matrix = np.loadtxt('./calib_matrix.csv', delimiter=',', dtype=str)
dist_coeffs   = np.loadtxt('./dist_coeff.csv', delimiter=',', dtype=str)

for f in images:
    img = cv2.imread(f)
    dst = cv2.undistort(img, camera_matrix, dist_coeffs)
    cv2.imwrite('./Images/Undistorted/' + f[9:-4] + '_undistorted.jpg', dst)

error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'undistort'
> Overload resolution failed:
>  - cameraMatrix data type = 19 is not supported
>  - Expected Ptr<cv::UMat> for argument 'cameraMatrix'
